In [1]:
import numpy as np
from matplotlib import pyplot as plt
from sklearn import preprocessing
import wfdb
import copy as cp
import scipy.signal as signal
import pickle
from sklearn import preprocessing
from tqdm import tqdm
import os
import re
import pandas as pd
import csv

from sklearn.ensemble import GradientBoostingClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from time import time
import timeit

OSError: dlopen(/Users/SAHIL/opt/anaconda3/lib/python3.8/site-packages/lightgbm/lib_lightgbm.so, 6): Library not loaded: /usr/local/opt/libomp/lib/libomp.dylib
  Referenced from: /Users/SAHIL/opt/anaconda3/lib/python3.8/site-packages/lightgbm/lib_lightgbm.so
  Reason: image not found

In [ ]:
record_list = [] # Initialize the array that will hold the list of our records

records = 'mit-bih-dataframes/subject_list.csv' # Get our record list like we did in the initial extraction
with open(records) as rfile:# Load our records into the array
    for record in rfile:
        record = record[0:-1] # The -1 removes the newline ("\n") character from the string
        record_list.append(record)

In [ ]:
dfdic = {}
for idx, x in enumerate(record_list):
    dfdic[x] = pd.read_csv('mit-bih-features/'+x+ '.csv', index_col=0)

In [ ]:
subject_df = pd.DataFrame()
for idx, x in enumerate(record_list):
    subject_df = pd.concat([subject_df, dfdic[x]])
subject_df = subject_df.drop(["Unnamed: 0.1"], axis=1)

In [ ]:
subject_df['Mappedrhythmlabels'] = subject_df['rhythmLabel'].map({'Normal':0, 'Other':0, 'AFIB':1})

In [ ]:
subject_df.head()

In [ ]:
statgboost_dic =  {
    'Run Time': [],
    'Accuracy': [],
    'Standard Error': [],
    'Sensitivity': [],
    'Specificity': [],
    'Precision': [],
    'F1-Score': []
}

In [ ]:
indaccs_dic = {}

In [ ]:
#Light Gradient Boosting 
LGBresults_lr1 = []
X=subject_df.iloc[:, 2:17]
Y=subject_df.iloc[:, -1]
for r in range(-3, 2):
    model=LGBMClassifier(n_estimators=500, learning_rate=10**r, max_depth=4, random_state=3)
    scores = cross_val_score(model, X, Y, cv=crossvalidation)
    print('Learning rate: {}'.format(10**r))
    print()
    print('Accuracy of each fold: \n {}'.format(scores))
    print("Avg accuracy: {}".format(scores.mean()))
    print()
    LGBresults_lr1.append(scores.mean())
print(LGBresults_lr1)
    

In [ ]:
#Light Gradient Boosting
LGBresults_lr2 = []
X=subject_df.iloc[:, 2:17]
Y=subject_df.iloc[:, -1]
for r in range(1, 10):
    model=LGBMClassifier(n_estimators=500, learning_rate=0.1*r, max_depth=4, random_state=3)
    scores = cross_val_score(model, X, Y, cv=crossvalidation)
    print('Learning rate: {}'.format(0.1*r))
    print()
    print('Accuracy of each fold: \n {}'.format(scores))
    print("Avg accuracy: {}".format(scores.mean()))
    print()
    LGBresults_lr2.append(scores.mean())
print(LGBresults_lr2)

In [ ]:
# Light Gradient Boosting
XGBresults_md = []
X=subject_df.iloc[:, 2:17]
Y=subject_df.iloc[:, -1]
for d in range(1, 26):
    model=LGBMClassifier(n_estimators=500, learning_rate=0.1, max_depth=d, random_state=3)
    scores = cross_val_score(model, X, Y, cv=crossvalidation)
    print('Max depth: {}'.format(d))
    print()
    print('Accuracy of each fold: \n {}'.format(scores))
    print("Avg accuracy: {}".format(scores.mean()))
    print()
    XGBresults_md.append(scores.mean())
print(XGBresults_md)

In [ ]:
#Extreme Gradient Boosting 
GBresults_lr1 = []
X=subject_df.iloc[:, 2:17]
Y=subject_df.iloc[:, -1]
for r in range(-3, 2):
    model=XGBClassifier(n_estimators=500, learning_rate=10**r, max_depth=4, random_state=3)
    scores = cross_val_score(model, X, Y, cv=crossvalidation)
    print('Learning rate: {}'.format(10**r))
    print()
    print('Accuracy of each fold: \n {}'.format(scores))
    print("Avg accuracy: {}".format(scores.mean()))
    print()
    GBresults_lr1.append(scores.mean())
print(GBresults_lr1)

In [ ]:
# Extreme Gradient Boosting
GBresults_lr2 = []
X=subject_df.iloc[:, 2:17]
Y=subject_df.iloc[:, -1]
for r in range(1, 10):
    model=XGBClassifier(n_estimators=500, learning_rate=0.1*r, max_depth=4, random_state=3)
    scores = cross_val_score(model, X, Y, cv=crossvalidation)
    print('Learning rate: {}'.format(0.1*r))
    print()
    print('Accuracy of each fold: \n {}'.format(scores))
    print("Avg accuracy: {}".format(scores.mean()))
    print()
    GBresults_lr2.append(scores.mean())
print(GBresults_lr2)

In [ ]:
# Extreme Gradient Boosting
GBresults_md = []
X=subject_df.iloc[:, 2:17]
Y=subject_df.iloc[:, -1]
for d in range(1, 26):
    model=XGBClassifier(n_estimators=500, learning_rate=0.1, max_depth=d, random_state=3)
    scores = cross_val_score(model, X, Y, cv=crossvalidation)
    print('Max depth: {}'.format(d))
    print()
    print('Accuracy of each fold: \n {}'.format(scores))
    print("Avg accuracy: {}".format(scores.mean()))
    print()
    XGBresults_md.append(scores.mean())
print(XGBresults_md)

In [ ]:
#Light Gradient Boosting
kf=KFold(n_splits=5, random_state=3, shuffle=True)
model = LGBMClassifier(n_estimators=500, learning_rate=0.1, max_depth=8, random_state=3)

start_time = timeit.default_timer()

acc_score = []
Truth = []
Output = []

for train_index, test_index in kf.split(subject_df):
    X_train = subject_df.iloc[train_index, 2:17]
    X_test = subject_df.iloc[test_index, 2:17]
    Y_train = subject_df.iloc[train_index, -1]
    Y_test = subject_df.iloc[test_index, -1]

    model.fit(X_train, Y_train)
    pred_values = model.predict(X_test)

    acc = accuracy_score(Y_test, pred_values)
    acc_score.append(acc)

    Truth.extend(Y_test.values.reshape(Y_test.shape[0])) 
    Output.extend(pred_values)

elapsed = timeit.default_timer() - start_time



print("---Run time is %s seconds ---" % elapsed)
print()
print('Accuracy of each fold: \n {}'.format(acc_score))
print("Avg accuracy: {}".format(np.mean(acc_score)))
print('Std of accuracy : \n{}'.format(np.std(acc_score)))
print()
print(confusion_matrix(Truth, Output))
print()
print(classification_report(Truth, Output))

cm = confusion_matrix(Truth, Output)

sensitivity = cm[0][0]/(cm[0][0]+cm[0][1])
specificity = cm[1][1]/(cm[1][0]+cm[1][1])
precision = (cm[0][0])/(cm[0][0]+cm[1][0])
f1_score = (2*precision*sensitivity)/(precision+sensitivity)

print(sensitivity)
print(specificity)
print(precision)
print(f1_score)

statgboost_dic['Run Time'].append(elapsed)
statgboost_dic['Accuracy'].append(np.mean(acc_score))
statgboost_dic['Standard Error'].append(np.std(acc_score))
statgboost_dic['Sensitivity'].append(sensitivity)
statgboost_dic['Specificity'].append(specificity)
statgboost_dic['Precision'].append(precision)
statgboost_dic['F1-Score'].append(f1_score)

indaccs_dic['LGBM']=acc_score

In [ ]:
#Extreme Gradient Boosting 
kf=KFold(n_splits=5, random_state=3, shuffle=True)
model = XGBClassifier(n_estimators=500, learning_rate=0.1, max_depth=8, random_state=3)

start_time = timeit.default_timer(n_estimators=500, learning_rate=0.1, max_depth=8, random_state=3)

acc_score = []
Truth = []
Output = []

for train_index, test_index in kf.split(subject_df):
    X_train = subject_df.iloc[train_index, 2:17]
    X_test = subject_df.iloc[test_index, 2:17]
    Y_train = subject_df.iloc[train_index, -1]
    Y_test = subject_df.iloc[test_index, -1]

    model.fit(X_train, Y_train)
    pred_values = model.predict(X_test)

    acc = accuracy_score(Y_test, pred_values)
    acc_score.append(acc)

    Truth.extend(Y_test.values.reshape(Y_test.shape[0])) 
    Output.extend(pred_values)

elapsed = timeit.default_timer() - start_time



print("---Run time is %s seconds ---" % elapsed)
print()
print('Accuracy of each fold: \n {}'.format(acc_score))
print("Avg accuracy: {}".format(np.mean(acc_score)))
print('Std of accuracy : \n{}'.format(np.std(acc_score)))
print()
print(confusion_matrix(Truth, Output))
print()
print(classification_report(Truth, Output))

cm = confusion_matrix(Truth, Output)

sensitivity = cm[0][0]/(cm[0][0]+cm[0][1])
specificity = cm[1][1]/(cm[1][0]+cm[1][1])
precision = (cm[0][0])/(cm[0][0]+cm[1][0])
f1_score = (2*precision*sensitivity)/(precision+sensitivity)

print(sensitivity)
print(specificity)
print(precision)
print(f1_score)

statgboost_dic['Run Time'].append(elapsed)
statgboost_dic['Accuracy'].append(np.mean(acc_score))
statgboost_dic['Standard Error'].append(np.std(acc_score))
statgboost_dic['Sensitivity'].append(sensitivity)
statgboost_dic['Specificity'].append(specificity)
statgboost_dic['Precision'].append(precision)
statgboost_dic['F1-Score'].append(f1_score)

indaccs_dic['XGBoost']=acc_score

In [ ]:
gboostdf = pd.DataFrame(
    data = np.transpose(np.array([
                                statgboost_dic["Accuracy"],
                                statgboost_dic["Standard Error"],
                                statgboost_dic["Sensitivity"],
                                statgboost_dic["Specificity"],
                                statgboost_dic["Precision"],
                                statgboost_dic["F1-Score"],
                                statgboost_dic["Run Time"] 
                            ])
                       ),
    
    columns = ["Accuracy", "Standard Error", "Sensitivity", "Specificity", "Precision", "F1-Score", "Run Time"]
    
)

gboostdf.set_index(pd.Index(["LGBoost", "XGBoost"]))